In [ ]:
import torch , time
from transformers import AutoTokenizer, AutoModelForCausalLM

def chat(model_path , prompt):

    start = time.time()
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.float16,  # works for 8-bit (it will auto-handle quantization)
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    
    prompt_model = (
            "<start_of_turn>user\n"
            + prompt
            + "<end_of_turn>\n<start_of_turn>model\n"
        )

    inputs = tokenizer(prompt_model, return_tensors="pt").to(model.device)

    outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    reply = tokenizer.decode(outputs[0], skip_special_tokens=False)
        
    # # Extract only model part
    if "<start_of_turn>model" in reply:
        reply = reply.split("<start_of_turn>model")[-1]
    if "<end_of_turn>" in reply:
        reply = reply.split("<end_of_turn>")[0]

    print("\nUser:" , prompt)
    print("\nModel:", reply.strip())
    print(f"\n 🤖 This response took {- start + time.time()} seconds to process")





In [ ]:
model = "anxity"

In [ ]:
while True:
    prompt = input("\nYou: ")
    
    if prompt == "quit" or prompt == "exit": break
    
    chat(f"results/{model}" , prompt)   

print("😁 WE HOPE YOU ENJOYED !!! ")